# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv from WeatherPy
cities_csv = "output_data/WeatherPy/cities.csv"

cities_weather = pd.read_csv(cities_csv, index_col=False)
cities_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kavieng,100,PG,1665538862,61,-2.5744,150.7967,85.95,5.46
1,1,Kavaratti,100,IN,1665538862,74,10.5669,72.6420,81.21,7.05
2,2,Carnarvon,48,AU,1665538862,46,-24.8667,113.6333,65.14,25.90
3,3,Piaçabuçu,6,BR,1665538862,82,-10.4056,-36.4344,74.37,10.47
4,4,Bethel,0,US,1665538862,80,41.3712,-73.4140,59.65,1.01
...,...,...,...,...,...,...,...,...,...,...
496,496,Dombås,87,NO,1665538952,88,62.0696,9.1235,30.15,3.78
497,497,Kharp,78,RU,1665538998,93,66.8014,65.8081,36.36,6.08
498,498,Peace River,40,CA,1665538998,40,56.2501,-117.2860,52.11,3.44
499,499,Rochester,0,US,1665538998,72,43.1548,-77.6156,61.14,0.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [30]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_weather[["Lat", "Lng"]].astype(float)

# Store 'Humidity' into weights
weights = cities_weather["Humidity"].astype(float)

In [13]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create a new dataframe with the cities filtered for the following conditions:
# Max temp inbetween 70 and 80F, Windspeed less than 10 mph and 0% Cloudiness.
cities_weather_fit = cities_weather[(cities_weather["Max Temp"] < 80) & (cities_weather["Max Temp"] > 70)
                                                                      & (cities_weather["Wind Speed"] < 10)
                                                                      & (cities_weather["Cloudiness"] == 0) ]
cities_weather_fit

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
83,83,Salalah,0,OM,1665538881,49,17.0151,54.0924,73.49,2.30
161,161,Iranshahr,0,IR,1665538781,10,27.2025,60.6848,70.34,4.50
226,226,Assiut,0,EG,1665538922,55,27.1810,31.1837,70.41,8.39
344,344,Akhmīm,0,EG,1665538957,47,26.5622,31.7450,70.79,8.37
360,360,Sakakah,0,SA,1665538675,32,29.9697,40.2064,71.17,5.77
391,391,Kiryat Gat,0,IL,1665538970,100,31.6100,34.7642,71.40,3.04
418,418,Riyadh,0,SA,1665538914,20,24.6877,46.7219,71.74,5.73
434,434,Roura,0,GF,1665538982,94,4.7263,-52.3274,78.71,0.00
449,449,Bushehr,0,IR,1665538928,61,28.9684,50.8385,77.09,0.00
490,490,Truth or Consequences,0,US,1665538997,46,33.1284,-107.2528,71.83,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store the filtered cities data into a variable named hotel_df.
hotel_df = cities_weather_fit
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
83,83,Salalah,0,OM,1665538881,49,17.0151,54.0924,73.49,2.30
161,161,Iranshahr,0,IR,1665538781,10,27.2025,60.6848,70.34,4.50
226,226,Assiut,0,EG,1665538922,55,27.1810,31.1837,70.41,8.39
344,344,Akhmīm,0,EG,1665538957,47,26.5622,31.7450,70.79,8.37
360,360,Sakakah,0,SA,1665538675,32,29.9697,40.2064,71.17,5.77
391,391,Kiryat Gat,0,IL,1665538970,100,31.6100,34.7642,71.40,3.04
418,418,Riyadh,0,SA,1665538914,20,24.6877,46.7219,71.74,5.73
434,434,Roura,0,GF,1665538982,94,4.7263,-52.3274,78.71,0.00
449,449,Bushehr,0,IR,1665538928,61,28.9684,50.8385,77.09,0.00
490,490,Truth or Consequences,0,US,1665538997,46,33.1284,-107.2528,71.83,5.75


In [6]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
83,83,Salalah,0,OM,1665538881,49,17.0151,54.0924,73.49,2.30,
161,161,Iranshahr,0,IR,1665538781,10,27.2025,60.6848,70.34,4.50,
226,226,Assiut,0,EG,1665538922,55,27.1810,31.1837,70.41,8.39,
344,344,Akhmīm,0,EG,1665538957,47,26.5622,31.7450,70.79,8.37,
360,360,Sakakah,0,SA,1665538675,32,29.9697,40.2064,71.17,5.77,
391,391,Kiryat Gat,0,IL,1665538970,100,31.6100,34.7642,71.40,3.04,
418,418,Riyadh,0,SA,1665538914,20,24.6877,46.7219,71.74,5.73,
434,434,Roura,0,GF,1665538982,94,4.7263,-52.3274,78.71,0.00,
449,449,Bushehr,0,IR,1665538928,61,28.9684,50.8385,77.09,0.00,
490,490,Truth or Consequences,0,US,1665538997,46,33.1284,-107.2528,71.83,5.75,


In [24]:
# find a hotel within 5000m radius for each city's coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up the required parameters
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
#   get lat and lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
#   add location to params dict
    params["location"] = f"{lat},{lng}"
    
#   assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#   extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
       
    except (KeyError, IndexError):
        print("No hotel at {row['City']}")
        
    print("------------")


Closest hotel in Salalah is Salalah Gardens Hotel.
------------
Closest hotel in Iranshahr is هتل و رستوران قصر.
------------
Closest hotel in Assiut is Nile Goddess Hotel الفندق العائم نايل جودس.
------------
Closest hotel in Akhmīm is Diamond Azur Hotel.
------------
Closest hotel in Sakakah is Raoum Inn Hotel.
------------
Closest hotel in Kiryat Gat is Desert Gat.
------------
Closest hotel in Riyadh is Le Park Concord Rabwah Riyadh.
------------
Closest hotel in Roura is Rour'Attitude.
------------
Closest hotel in Bushehr is Hotel busherr staying.
------------
Closest hotel in Truth or Consequences is Holiday Inn Express & Suites Truth or Consequences, an IHG Hotel.
------------


In [25]:
# Display the filtered cities along with the hotel names
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
83,83,Salalah,0,OM,1665538881,49,17.0151,54.0924,73.49,2.30,Salalah Gardens Hotel
161,161,Iranshahr,0,IR,1665538781,10,27.2025,60.6848,70.34,4.50,هتل و رستوران قصر
226,226,Assiut,0,EG,1665538922,55,27.1810,31.1837,70.41,8.39,Nile Goddess Hotel الفندق العائم نايل جودس
344,344,Akhmīm,0,EG,1665538957,47,26.5622,31.7450,70.79,8.37,Diamond Azur Hotel
360,360,Sakakah,0,SA,1665538675,32,29.9697,40.2064,71.17,5.77,Raoum Inn Hotel
391,391,Kiryat Gat,0,IL,1665538970,100,31.6100,34.7642,71.40,3.04,Desert Gat
418,418,Riyadh,0,SA,1665538914,20,24.6877,46.7219,71.74,5.73,Le Park Concord Rabwah Riyadh
434,434,Roura,0,GF,1665538982,94,4.7263,-52.3274,78.71,0.00,Rour'Attitude
449,449,Bushehr,0,IR,1665538928,61,28.9684,50.8385,77.09,0.00,Hotel busherr staying
490,490,Truth or Consequences,0,US,1665538997,46,33.1284,-107.2528,71.83,5.75,Holiday Inn Express & Suites Truth or Conseque...


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer on top of the humidity heat map created earlier.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))